In [66]:
import numpy as np
np.set_printoptions(precision=2,suppress=True)
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.stats import zscore

import statsmodels.api as sm
import statsmodels
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.api import ARDL

In [67]:
data = pd.read_csv('Genshin Impact_monthly_ARDL.csv',index_col=0,parse_dates=True)
data

,Android DAU,Apple DAU,Android installs,Apple installs,Android IAP,Apple iAP,Android DAU(pct),Apple DAU(pct)
event_date,,,,,,,,
2020-02-29,319618.6483,295615.4586,1250609.50,688867.200,2581095.00,6723387.60,NaN,NaN
2020-03-31,429316.5419,305261.3032,2594833.50,697877.100,3193806.00,8583826.20,0.343215,0.032630
2020-04-30,413043.0933,277208.7300,1957455.50,541344.600,2824782.40,6384990.78,-0.037905,-0.091897
2020-05-31,376833.4452,270917.1290,1674760.50,576225.900,2448102.60,6327327.14,-0.087666,-0.022696
2020-06-30,350617.3867,260290.5300,1402005.50,556896.600,2458328.40,6580056.40,-0.069569,-0.039225
2020-07-31,392962.7871,258283.4806,2174819.50,574230.600,2313300.60,5480158.84,0.120774,-0.007711
2020-08-31,353379.6129,236682.6097,1351101.50,460282.500,2512374.60,5749746.25,-0.100730,-0.083632
2020-09-30,351057.9200,233996.4000,1491273.00,518853.600,3005724.90,7010652.46,-0.006570,-0.011349
2020-10-31,327436.0774,218535.9097,1337903.00,393292.800,3375662.00,8290427.41,-0.067288,-0.066071


In [68]:
data_train = data.loc['2020-02-29':'2021-01-31']
data_train[['Android DAU(pct)_t+1','Apple DAU(pct)_t+1']]= data_train[['Android DAU(pct)','Apple DAU(pct)']].shift(-1)
data_train = data_train.iloc[:-1]
data_train

/var/folders/7l/j9np0sbx1b33249mt44v25980000gn/T/ipykernel_6773/1745314223.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train[['Android DAU(pct)_t+1','Apple DAU(pct)_t+1']]= data_train[['Android DAU(pct)','Apple DAU(pct)']].shift(-1)
/var/folders/7l/j9np0sbx1b33249mt44v25980000gn/T/ipykernel_6773/1745314223.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train[['Android DAU(pct)_t+1','Apple DAU(pct)_t+1']]= data_train[['Android DAU(pct)','Apple DAU(pct)']].shift(-1)


,Android DAU,Apple DAU,Android installs,Apple installs,Android IAP,Apple iAP,Android DAU(pct),Apple DAU(pct),Android DAU(pct)_t+1,Apple DAU(pct)_t+1
event_date,,,,,,,,,,
2020-02-29,319618.6483,295615.4586,1250609.5,688867.2,2581095.0,6723387.60,NaN,NaN,0.343215,0.032630
2020-03-31,429316.5419,305261.3032,2594833.5,697877.1,3193806.0,8583826.20,0.343215,0.032630,-0.037905,-0.091897
2020-04-30,413043.0933,277208.7300,1957455.5,541344.6,2824782.4,6384990.78,-0.037905,-0.091897,-0.087666,-0.022696
2020-05-31,376833.4452,270917.1290,1674760.5,576225.9,2448102.6,6327327.14,-0.087666,-0.022696,-0.069569,-0.039225
2020-06-30,350617.3867,260290.5300,1402005.5,556896.6,2458328.4,6580056.40,-0.069569,-0.039225,0.120774,-0.007711
2020-07-31,392962.7871,258283.4806,2174819.5,574230.6,2313300.6,5480158.84,0.120774,-0.007711,-0.100730,-0.083632
2020-08-31,353379.6129,236682.6097,1351101.5,460282.5,2512374.6,5749746.25,-0.100730,-0.083632,-0.006570,-0.011349
2020-09-30,351057.9200,233996.4000,1491273.0,518853.6,3005724.9,7010652.46,-0.006570,-0.011349,-0.067288,-0.066071
2020-10-31,327436.0774,218535.9097,1337903.0,393292.8,3375662.0,8290427.41,-0.067288,-0.066071,-0.008402,0.008358


# forcasting Android DAU

In [71]:
model = ARDL(
    data_train[['Android DAU(pct)_t+1']],
    1,
    data_train[['Android installs','Android IAP']],
    {
        'Android installs':1,
        'Android IAP':0
    },
).fit(cov_type='HAC',cov_kwds={'maxlags':3})

print(model.summary())

                               ARDL Model Results                               
Dep. Variable:     Android DAU(pct)_t+1   No. Observations:                   11
Model:                    ARDL(1, 1, 0)   Log Likelihood                  17.342
Method:                 Conditional MLE   S.D. of innovations              0.043
Date:                  Sun, 18 Dec 2022   AIC                            -22.684
Time:                          15:42:26   BIC                            -20.869
Sample:                      03-31-2020   HQIC                           -24.676
                           - 12-31-2020                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                       0.5486      0.128      4.290      0.008       0.220       0.877
Android DAU(pct)_t+1.L1     0.9174      0.155      5.904      0.002       0.

/Users/yanyan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)


In [72]:
forcasts = {}

In [73]:
forcasts.update({
    'Android DAU(f)': model.predict(exog_oos=data[['Android installs','Android IAP']],dynamic=True,start=11,end=23).shift(1).dropna()
})

/Users/yanyan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)


# forcasting Apple DAU

In [74]:
model_Apple_DAU = ARDL(
    data_train[['Apple DAU(pct)_t+1']],
    1,
    data_train[['Apple installs','Apple iAP']],
    {
        'Apple installs':0,
        'Apple iAP':0
    },
).fit(cov_type='HAC',cov_kwds={'maxlags':3})

print(model_Apple_DAU.summary())

                              ARDL Model Results                              
Dep. Variable:     Apple DAU(pct)_t+1   No. Observations:                   11
Model:                  ARDL(1, 0, 0)   Log Likelihood                  26.242
Method:               Conditional MLE   S.D. of innovations              0.018
Date:                Sun, 18 Dec 2022   AIC                            -42.484
Time:                        15:42:33   BIC                            -40.971
Sample:                    03-31-2020   HQIC                           -44.143
                         - 12-31-2020                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     0.0495      0.064      0.768      0.472      -0.108       0.207
Apple DAU(pct)_t+1.L1    -0.3554      0.152     -2.345      0.057      -0.726       0.015
Apple in

/Users/yanyan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)


In [75]:
forcasts.update({
    'Apple DAU(f)': model_Apple_DAU.predict(exog_oos=data[['Apple installs','Apple iAP']],dynamic=True,start=11,end=23).shift(1).dropna()
})

/Users/yanyan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)


In [76]:
forcasts_pd = (pd.DataFrame(forcasts)+1)*np.array([348049.341900,214980.880600])

In [79]:
forcasts_pd

,Android DAU(f),Apple DAU(f)
2021-02-28,401307.973568,196979.435684
2021-03-31,266185.107011,204502.073748
2021-04-30,253933.884774,207728.196650
2021-05-31,265908.914148,204724.554149
2021-06-30,300174.864193,207039.819651
2021-07-31,241366.618358,204232.932530
2021-08-31,294655.826999,211377.699801
2021-09-30,297820.850393,207065.160540
2021-10-31,314064.306302,216344.983065
2021-11-30,326374.857675,209193.029314


In [80]:
forcasts_pd.to_csv('ARDL_prediction.csv')